In [4]:
import pandas as pd
import spacy
import snorkel

# Reading in and working with the data
The few-NERD data is much more detailed than we need, if we want to make a direct comparison to the silver data we have previously worked on. Therefore, we need to read in the few-NERD data and convert it to our preferred format (with labels O-6). We then save that in a new csv file so we don't have to convert everything every time we want to work with the data. In the future, only the already converted csv will be read in ;-)
```
data=pd.read_csv("gold-to-silver/supervised/train.txt", sep="\t", header=None, names=["Tokens", "Tags"])
started=False
for index, row in data.iterrows():
    tag=row.iloc[1]

    if tag=="O":
        started=False
    
    elif tag.startswith("art") or tag.startswith("product") or tag.startswith("event"):
        data.iloc[index, 1]="O"
        started=False
        
    elif tag.startswith("building") or tag.startswith("location"):
        if started==False:
            data.iloc[index, 1]="5"
            started=True
            
        else:
            data.iloc[index, 1]="6"
        
    elif tag.startswith("organization"):
        if started==False:
            data.iloc[index, 1]="3"
            started=True
            
        else:
            data.iloc[index, 1]="4"
            
    elif tag.startswith("person"):
        if started==False:
            data.iloc[index, 1]="1"
            started=True
        else:
            data.iloc[index, 1]="2"
            
    elif tag.startswith("other"):
        if tag=="other-god":
            if started==False:
                data.iloc[index, 1]="1"
                started=True
            else:
                data.iloc[index, 1]="2"
        else:
            data.iloc[index, 1]="O"
            started=False
            
    else:
        print("Weird!")
data.to_csv("gold-to-silver/fewnerd-supervised-training-data.csv", sep="\t", index=False)
```

In [18]:
with open("fewnerd-supervised-training-data.csv", encoding="utf-8") as file:
    train=file.read()
    
training_data_70=pd.read_csv("fewnerd-supervised-training-data.csv", sep="\t")
validation_data_10=pd.read_csv("fewnerd-supervised-validation-data.csv", sep="\t")
testing_data_20=pd.read_csv("fewnerd-supervised-testing-data.csv", sep="\t")

# With SpaCy
SpaCy recommends using their config file through the command line

In [10]:
from spacy.cli import download

download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
test123=train[0:1000000]

In [27]:
nlp=spacy.load("en_core_web_sm")
doc=nlp(test123)
for ent in doc.ents:
    print(ent.text, " | ", ent.label_)

Tokens  |  GPE
1  |  CARDINAL
1  |  CARDINAL
2  |  CARDINAL
1  |  CARDINAL
2  |  CARDINAL
multimillion  |  CARDINAL
2  |  CARDINAL
Seven  |  CARDINAL
1  |  CARDINAL
2  |  CARDINAL
1  |  CARDINAL
1  |  CARDINAL
2  |  CARDINAL
Pakistani  |  NORP
3  |  CARDINAL
Indian  |  NORP
February  |  DATE
2008  |  DATE
3  |  CARDINAL
4  |  CARDINAL
4  |  CARDINAL
UK  |  GPE
Texas  |  GPE
3  |  CARDINAL
4  |  CARDINAL
50  |  CARDINAL
3  |  CARDINAL
4  |  CARDINAL
1919  |  DATE
3  |  CARDINAL
First  |  ORDINAL
3  |  CARDINAL
4  |  CARDINAL
3  |  CARDINAL
1  |  CARDINAL
2003  |  DATE
100EX  |  DATE
2004  |  DATE
one  |  CARDINAL
four  |  CARDINAL
4  |  CARDINAL
second  |  ORDINAL
Iranian  |  NORP
1905-1911  |  DATE
French  |  NORP
Russian  |  NORP
3  |  CARDINAL
4  |  CARDINAL
1992  |  DATE
Two	3  |  CARDINAL
4  |  CARDINAL
4
Inc  |  MONEY
4  |  CARDINAL
4  |  CARDINAL
3  |  CARDINAL
4  |  CARDINAL
4
Inc  |  MONEY
4  |  CARDINAL
4  |  CARDINAL
3  |  CARDINAL
4  |  CARDINAL
Assyrians  |  NORP
two  |  CA

# With Snorkel
https://github.com/snorkel-team/snorkel

# With Python Spark
https://medium.com/john-snow-labs/named-entity-recognition-ner-with-python-at-scale-a37d7c241135